In [215]:
import pandas as pd
import numpy as np

# Importação e Cópia
Estou importando os dados diretamente do Gsheet, e na célula seguinte fazendo a cópia dos dataframes. A cópia serve para não ter o trabalho de importar dataframe da nuvem novamente.

In [319]:
DF_COMPRAS_2024 = pd.read_excel(
    "https://docs.google.com/spreadsheets/d/117UmGO31yywC6mucCwMFPmnQcg1kRI5WVRR1Cvpb6l8/export?format=xlsx&id=117UmGO31yywC6mucCwMFPmnQcg1kRI5WVRR1Cvpb6l8",
    sheet_name="Lançamento de NF Não Recorrente",
    usecols=[
        "Marca",
        "Unidade",
        "Valor Corrigido",
        "Mês Corrigido",
    ]
)

DF_DE_PARA_OFC = pd.read_excel(
    'https://docs.google.com/spreadsheets/d/1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ/export?=format=xlsx&id=1FXJHfBeenCm7kDq-3SpJjuuBY0nXZ2xF-6DTK2HQyaQ', 
    sheet_name="Unidades",
    usecols=[
        "Chave 2024",
        "Marca 2024",
        "Unidade 2024"
    ]
)

c:\Users\Fernando Santos\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Cell E1954 is marked as a date but the serial value 6692727.0 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():


In [320]:
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024.copy()
DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC.copy()

# Correção do Mês
Aqui estou covertendo os meses faltantes pelo mês atual, e não só isso, convertendo os valores para inteiro.

In [321]:
from datetime import datetime

DF_COMPRAS_2024_COPY["Mês Corrigido"] = pd.to_numeric(DF_COMPRAS_2024_COPY["Mês Corrigido"], "coerce")
DF_COMPRAS_2024_COPY["Mês Corrigido"] = DF_COMPRAS_2024_COPY["Mês Corrigido"].fillna(0)
DF_COMPRAS_2024_COPY["Mês Corrigido"] = DF_COMPRAS_2024_COPY["Mês Corrigido"].astype("int")
DF_COMPRAS_2024_COPY.loc[DF_COMPRAS_2024_COPY["Mês Corrigido"] == 0, "Mês Corrigido"] = datetime.now().month

# Correção de Marca e Unidade
Agora, vamos atualizar a marca e unidade para os nomes de 2024, nesse processo estou fazendo:
- Retirandos os dados faltantes
- Aplicando função ```stip()``` na Marca e Unidade, para retirar os espaços em branco
- Criando uma coluna apenas com a Unidade
- Criando a Chave 2024
- Aplicando o DePara e puxando as Marcas e Unidades atualizadas

In [322]:
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY[DF_COMPRAS_2024_COPY["Marca"].notna()]
DF_COMPRAS_2024_COPY = DF_COMPRAS_2024_COPY[DF_COMPRAS_2024_COPY["Unidade"].notna()]
DF_DE_PARA_OFC_COPY = DF_DE_PARA_OFC_COPY[DF_DE_PARA_OFC_COPY["Chave 2024"].notna()]

DF_COMPRAS_2024_COPY["Marca"] = DF_COMPRAS_2024_COPY.apply(lambda x: x["Marca"].strip(), axis=1)
DF_COMPRAS_2024_COPY["Unidade"] = DF_COMPRAS_2024_COPY.apply(lambda x: x["Unidade"].strip(), axis=1)

DF_COMPRAS_2024_COPY["Unidade Atualizada"] = DF_COMPRAS_2024_COPY.apply(lambda row: row["Unidade"].replace(f"{row['Marca']} ", ""), axis=1)
DF_COMPRAS_2024_COPY["Chave 2024"] = DF_COMPRAS_2024_COPY["Marca"] + DF_COMPRAS_2024_COPY["Unidade Atualizada"] 

In [323]:
DF_COMPRAS_2024_DE_PARA = pd.merge(DF_COMPRAS_2024_COPY, DF_DE_PARA_OFC_COPY, on="Chave 2024", how="left")
DF_COMPRAS_2024_DE_PARA.head()

,Marca,Unidade,Valor Corrigido,Mês Corrigido,Unidade Atualizada,Chave 2024,Marca 2024,Unidade 2024
0,Anglo Alante,Anglo Alante São José dos Campos,106020.0,11,São José dos Campos,Anglo AlanteSão José dos Campos,Anglo Alante,São José dos Campos
1,Elite,Elite Idesa Taubaté,40423.0,11,Idesa Taubaté,EliteIdesa Taubaté,Elite,Idesa Taubaté
2,Elite,Elite Idesa Taubaté,15146.0,11,Idesa Taubaté,EliteIdesa Taubaté,Elite,Idesa Taubaté
3,Anglo Alante,Anglo Alante São José dos Campos,21529.6,11,São José dos Campos,Anglo AlanteSão José dos Campos,Anglo Alante,São José dos Campos
4,Elite,Elite Taboão da Serra,3199.7,1,Taboão da Serra,EliteTaboão da Serra,Elite,Taboão da Serra


# Dividindo o DataFrame
Nesse momento estou divindo meu DataFrame em dois, um com os erros, no qual vou tratar e unir novamente ao dataset original.

In [324]:
DF_ERROS = DF_COMPRAS_2024_DE_PARA[DF_COMPRAS_2024_DE_PARA["Marca 2024"].isna()]
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[DF_COMPRAS_2024_DE_PARA["Marca 2024"].notna()]

In [325]:
DF_ERROS = DF_ERROS.loc[(DF_ERROS["Marca"] != "NUs")]
DF_ERROS = DF_ERROS.loc[(DF_ERROS["Marca"] != "Visita BE")]
DF_ERROS = DF_ERROS.loc[(DF_ERROS["Marca"] != "Salta")]

DF_ERROS["Marca"] = DF_ERROS.apply(lambda x: x["Unidade"].split(" ")[0], axis=1)

DF_ERROS["Unidade Atualizada"] = DF_ERROS.apply(lambda row: row["Unidade"].replace(f"{row['Marca']} ", ""), axis=1)
DF_ERROS["Chave 2024"] = DF_ERROS["Marca"] + DF_ERROS["Unidade Atualizada"]
DF_ERROS.drop(columns=[
    "Marca 2024",
    "Unidade 2024",
], inplace=True)

DF_ERROS = pd.merge(DF_ERROS, DF_DE_PARA_OFC_COPY, on="Chave 2024", how="left")

In [326]:
DF_ERROS[DF_ERROS["Marca 2024"].isna()]

,Marca,Unidade,Valor Corrigido,Mês Corrigido,Unidade Atualizada,Chave 2024,Marca 2024,Unidade 2024
2,Anglo,Anglo Alante Paulínia,1277.2,12,Alante Paulínia,AngloAlante Paulínia,NaN,NaN
17,PH,PH tijuca 1,5524.6,12,tijuca 1,PHtijuca 1,NaN,NaN
22,Elite,Elite SP Opção Mauá,1800.0,1,SP Opção Mauá,EliteSP Opção Mauá,NaN,NaN
23,Maxi,Maxi Cuiaba,799.0,1,Cuiaba,MaxiCuiaba,NaN,NaN
29,Elite\n,Elite\n Santo André,418.0,4,Santo André,Elite\nSanto André,NaN,NaN


# Unindo as Bases de Dados

In [327]:
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[[
    "Marca 2024",
    "Unidade 2024",
    "Valor Corrigido",
    "Mês Corrigido",
]]

DF_COMPRAS_2024_DE_PARA["Ano"] = 2024
DF_COMPRAS_2024_DE_PARA["NO"] = "Ativos"
DF_COMPRAS_2024_DE_PARA["Categoria"] = "Supply"
DF_COMPRAS_2024_DE_PARA["Tipo Valor"] = "Capex"
DF_COMPRAS_2024_DE_PARA["Tipo"] = "Real"

DF_ERROS_ATUALIZADOS = DF_ERROS[DF_ERROS["Marca 2024"].notna()]
DF_ERROS_ATUALIZADOS = DF_ERROS_ATUALIZADOS[[
    "Marca 2024",
    "Unidade 2024",
    "Valor Corrigido",
    "Mês Corrigido"
]]

DF_ERROS_ATUALIZADOS["Ano"] = 2024
DF_ERROS_ATUALIZADOS["NO"] = "Ativos"
DF_ERROS_ATUALIZADOS["Categoria"] = "Supply"
DF_ERROS_ATUALIZADOS["Tipo Valor"] = "Capex"
DF_ERROS_ATUALIZADOS["Tipo"] = "Real"

In [328]:
DF_COMPRAS_2024_DE_PARA = DF_COMPRAS_2024_DE_PARA[[
    "Valor Corrigido",
    "Marca 2024",
    "Unidade 2024",
    "Ano",
    "NO",
    "Categoria",
    "Tipo Valor",
    "Tipo",
    "Mês Corrigido",
]]

DF_ERROS_ATUALIZADOS = DF_ERROS_ATUALIZADOS[[
    "Valor Corrigido",
    "Marca 2024",
    "Unidade 2024",
    "Ano",
    "NO",
    "Categoria",
    "Tipo Valor",
    "Tipo",
    "Mês Corrigido",
]]

In [329]:
DF_ATIVOS = pd.concat([DF_COMPRAS_2024_DE_PARA, DF_ERROS_ATUALIZADOS], axis=0)
DF_ATIVOS.head()

,Valor Corrigido,Marca 2024,Unidade 2024,Ano,NO,Categoria,Tipo Valor,Tipo,Mês Corrigido
0,106020.0,Anglo Alante,São José dos Campos,2024,Ativos,Supply,Capex,Real,11
1,40423.0,Elite,Idesa Taubaté,2024,Ativos,Supply,Capex,Real,11
2,15146.0,Elite,Idesa Taubaté,2024,Ativos,Supply,Capex,Real,11
3,21529.6,Anglo Alante,São José dos Campos,2024,Ativos,Supply,Capex,Real,11
4,3199.7,Elite,Taboão da Serra,2024,Ativos,Supply,Capex,Real,1


In [332]:
DF_ATIVOS["Valor Corrigido"].sum()

11204695.780000001

In [331]:
DF_ATIVOS.to_excel("Ativos 2024 (Padronizado).xlsx", index=False)